# RAG for RFP Q&A Demo

In [ ]:
%pip install -qU langchain langchain-openai langchain-cohere

In [ ]:
%pip install -qU qdrant-client lark

In [ ]:
import os

import dotenv

dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")

In [ ]:
os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# List of CSV file paths
rfp_file_paths = [
    "datasets/rfp_genai_app_vendor_ID1.csv",
    "datasets/rfp_genai_app_vendor_ID2.csv",
    "datasets/rfp_genai_app_vendor_ID3.csv",
    "datasets/rfp_genai_app_vendor_ID4.csv",
    "datasets/rfp_genai_app_vendor_ID5.csv"
]

documents = []

# Iterate through each file path in the list
for file_path in rfp_file_paths:
    loader = CSVLoader(
        file_path=file_path,
        metadata_columns=["Area", "Project_Title", "Last_Accessed_At", "Requester", "Status"]
    )

    # Load documents from the current CSV file
    rfp_docs = loader.load()
    # Extend the main documents list with the loaded documents
    documents.extend(rfp_docs)

In [ ]:
documents[:5]

In [ ]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="RFPCollection"
)

In [ ]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [ ]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [ ]:
naive_retrieval_chain.invoke({"question" : "List the questions related to NIST?"})["response"].content